In [ ]:
import requests
import pandas as pd
import json
from dateutil import parser
from datetime import datetime, timezone
import os
import pickle

In [ ]:
API_KEY = "a7f5a55a86693e1890a435daa9e97277-9af069c4659def07a9ad2affb59cb44c"
ACCOUNT_ID = "101-004-26690988-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [ ]:
session = requests.Session()

In [ ]:
session.headers.update({
    "Authorization" : f"Bearer {API_KEY}" ,
    "Content-Type" : "application/json"
})

In [ ]:
# Define the start and end times with pandas, which can handle the conversion to RFC 3339
start_date = pd.to_datetime("01/01/2024", dayfirst=True).isoformat() + 'Z'
end_date = pd.to_datetime("01/02/2024", dayfirst=True).isoformat() + 'Z'

print(f"Window start: {start_date}, window end: {end_date}.")

Window start: 2024-01-01T00:00:00Z, window end: 2024-02-01T00:00:00Z.


In [ ]:
params = {"from": str(start_date), "to": str(end_date), "granularity": "M15", "price": "MBA"}

In [ ]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [ ]:
response = session.get(url, params=None, data=None, headers=None)

In [ ]:
response.status_code

200

In [ ]:
data = response.json()

In [ ]:
instruments_list = data['instruments']

In [ ]:
len(instruments_list)

123

In [ ]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [ ]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']


In [ ]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_i }


In [ ]:
instruments_dict['XAU_USD']

{'name': 'XAU_USD',
 'type': 'METAL',
 'displayName': 'Gold',
 'pipLocation': -2,
 'displayPrecision': 3,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.05'}

In [ ]:
with open("Data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [ ]:
def fetch_candles(pair_name, start_date, end_date, granularity="M15"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = {"from": str(start_date), "to": str(end_date), "granularity": "M15", "price": "MBA"}

    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    final_data = []
    for candle in data:
        if candle['complete'] == False:
                continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])

        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df
def create_data_file(pair_name, start_date, end_date, granularity="M15"):
    code, data = fetch_candles(pair_name, start_date, end_date, granularity="M15")
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"Data/{pair_name}_{granularity}_1.pk1")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [ ]:
code, data = fetch_candles("XAU_USD", start_date, end_date, granularity="M15")

if code == 200 and data:
    candles_df = get_candles_df(data)
    # You can now work with `candles_df` DataFrame
else:
    print(f"Failed to fetch candles with status code: {code}")

In [ ]:
code

200

In [ ]:
len(data)

2018

In [ ]:
data[0]

{'complete': True,
 'volume': 397,
 'time': '2024-01-01T23:00:00.000000000Z',
 'bid': {'o': '2064.400', 'h': '2066.495', 'l': '2063.040', 'c': '2065.620'},
 'mid': {'o': '2065.845', 'h': '2068.995', 'l': '2064.135', 'c': '2065.865'},
 'ask': {'o': '2067.290', 'h': '2071.495', 'l': '2064.620', 'c': '2066.110'}}

In [ ]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-01-01 23:00:00+00:00,397,2065.845,2068.995,2064.135,2065.865,2064.40,2066.495,2063.04,2065.62,2067.29,2071.495,2064.62,2066.11
1,2024-01-01 23:15:00+00:00,498,2065.870,2066.295,2065.235,2065.275,2065.63,2066.080,2065.01,2065.07,2066.11,2066.540,2065.43,2065.48
2,2024-01-01 23:30:00+00:00,457,2065.225,2065.535,2064.300,2064.620,2065.07,2065.360,2064.14,2064.46,2065.38,2065.770,2064.46,2064.78
3,2024-01-01 23:45:00+00:00,554,2064.560,2064.590,2063.250,2063.795,2064.40,2064.430,2063.10,2063.62,2064.72,2064.750,2063.40,2063.97
4,2024-01-02 00:00:00+00:00,488,2063.835,2065.035,2063.475,2064.105,2063.67,2064.880,2063.31,2063.95,2064.00,2065.190,2063.64,2064.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,2024-01-31 21:45:00+00:00,1031,2038.425,2039.200,2036.685,2039.155,2038.18,2038.950,2036.42,2038.93,2038.67,2039.450,2036.95,2039.38
2014,2024-01-31 23:00:00+00:00,491,2040.025,2041.385,2039.880,2040.980,2039.63,2041.150,2039.50,2040.67,2040.42,2041.670,2040.10,2041.29
2015,2024-01-31 23:15:00+00:00,562,2040.970,2041.385,2040.115,2041.275,2040.65,2041.150,2039.81,2041.04,2041.29,2041.620,2040.36,2041.51
2016,2024-01-31 23:30:00+00:00,329,2041.280,2041.280,2040.100,2040.535,2041.05,2041.050,2039.92,2040.35,2041.51,2041.510,2040.28,2040.72


In [ ]:
create_data_file('XAU_USD', start_date, end_date, granularity='M15')

XAU_USD M15 2018 candles, 2024-01-01 23:00:00+00:00 2024-01-31 23:45:00+00:00


In [ ]:
# Set the directory where your .pk1 files are located
data_directory = 'Data'
# Set the directory where the new .pk1 file will be saved
augmented_directory = 'Data Augmented'


# List to hold dataframes
dfs = []

# Loop through the numbers 1 to 12 to read the files in order
for month in range(1, 13):
    # Construct the file name
    file_name = f'XAU_USD_M15_{month}.pk1'
    file_path = os.path.join(data_directory, file_name)

    # Check if the file exists
    if os.path.exists(file_path):
        # Load the DataFrame from a pickle file
        df = pd.read_pickle(file_path)
        dfs.append(df)
    else:
        print(f"File not found: {file_path}")

# Concatenate all the dataframes
full_year_df = pd.concat(dfs, ignore_index=True)

# Save the concatenated DataFrame to the new 'augmented_data' directory
full_year_file_name = 'XAU_USD_M15_2023.pk1'
full_year_file_path = os.path.join(augmented_directory, full_year_file_name)
full_year_df.to_pickle(full_year_file_path)

print(f"Concatenated DataFrame saved to {full_year_file_path}")

File not found: Data\XAU_USD_M15_2.pk1
File not found: Data\XAU_USD_M15_3.pk1
File not found: Data\XAU_USD_M15_4.pk1
File not found: Data\XAU_USD_M15_5.pk1
File not found: Data\XAU_USD_M15_6.pk1
File not found: Data\XAU_USD_M15_7.pk1
File not found: Data\XAU_USD_M15_8.pk1
File not found: Data\XAU_USD_M15_9.pk1
File not found: Data\XAU_USD_M15_10.pk1
File not found: Data\XAU_USD_M15_11.pk1
File not found: Data\XAU_USD_M15_12.pk1
Concatenated DataFrame saved to Data Augmented\XAU_USD_M15_2023.pk1


In [ ]:
# Set the file name of the new .pk1 file
file_name = 'XAU_USD_M15_2023.pk1'
# Construct the full path to the .pk1 file
file_path = os.path.join(augmented_directory, file_name)

# Check if the file exists
if os.path.exists(file_path):
    # Load the DataFrame from the pickle file
    df = pd.read_pickle(file_path)

    # Display the first few rows of the DataFrame
    print("The first few rows of the DataFrame:")
    print(df.head())

    # Display the last few rows of the DataFrame
    print("\nThe last few rows of the DataFrame:")
    print(df.tail())
else:
    print(f"The file does not exist at the specified path: {file_path}")

The first few rows of the DataFrame:
                       time  volume     mid_o     mid_h     mid_l     mid_c  \
0 2024-01-01 23:00:00+00:00     397  2065.845  2068.995  2064.135  2065.865   
1 2024-01-01 23:15:00+00:00     498  2065.870  2066.295  2065.235  2065.275   
2 2024-01-01 23:30:00+00:00     457  2065.225  2065.535  2064.300  2064.620   
3 2024-01-01 23:45:00+00:00     554  2064.560  2064.590  2063.250  2063.795   
4 2024-01-02 00:00:00+00:00     488  2063.835  2065.035  2063.475  2064.105   

     bid_o     bid_h    bid_l    bid_c    ask_o     ask_h    ask_l    ask_c  
0  2064.40  2066.495  2063.04  2065.62  2067.29  2071.495  2064.62  2066.11  
1  2065.63  2066.080  2065.01  2065.07  2066.11  2066.540  2065.43  2065.48  
2  2065.07  2065.360  2064.14  2064.46  2065.38  2065.770  2064.46  2064.78  
3  2064.40  2064.430  2063.10  2063.62  2064.72  2064.750  2063.40  2063.97  
4  2063.67  2064.880  2063.31  2063.95  2064.00  2065.190  2063.64  2064.26  

The last few rows o